# Тренировка YOLOv8

Этот ноутбук содержит код тренировки модели распознования объектов YOLOv8 на датасете, собранном из предоставленных компанией Самолёт данных. Тренировка производится в два этапа: в первом модель тренируется на объектах редких классов, после чего натренированные веса используются для тренировки на всех классах. Всего выделено 18 классов объектов:

Название класса | Идентификатор класса
-- | --
Ванная | bathtud
Дверь | door
Дверной проход и дверь | door-way
Отделанный потолок | finished ceiling
Отделанный пол | finished floor
Отделанная стена | finished wall 
Отделанный подоконник | finished windowsill
Мусор | garbage
Стянутый пол | started floor
Кухонная мебель | kitchen
Дверной проход без дверей | no-door-doorway
Радиатор отопления | radiator
Потолок без отделки | raw ceiling
Пол без отделки | raw floor
Стена без отделки | raw wall
Отверстие под окно | raw window
Раковина | sink
Розетка | socket
Стена со штукатуркой | started wall
Переключатель |switch
Унитаз | toilet seat
Окно | window
Подоконник | windowsill

In [4]:
ROBOFLOW_API_KEY = "nIU5i0GphiPWRZ3cOHGG"  # API ключ к сервису Roboflow
# Название рабочего пространства
ROBOFLOW_WORKSPACE = "applied-machine-learning-assignment-i-innopolis-university"
ROBOFLOW_PROJECT = "adssadsa"  # Название проекта в сервисе Roboflow 
# Название эксперимента соотв. первому этапу
FIRST_STAGE_EXPERIMENT_NAME = "unrepresentative_stage"  
# Название эксперимента соотв. второму этапу
SECOND_STAGE_EXPERIMENT_NAME = "all_classes_stage"  
ROBOFLOW_FIRST_STAGE_DATASET_VERSION = "6"
ROBOFLOW_SECOND_STAGE_DATASET_VERSION = "5"

YOLO_INITIAL_MODEL = "yolov8x.pt"

## Подготовка зависимостей

Для обучения используется модель YOLOv8 на PyTorch и датасет, размеченный на сервисе Roboflow. Для установки всех зависимостей используйте ячейку ниже:

In [2]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
# %pip install roboflow ultralytics

# Первый этап

Данные, предоставленные Самолётом, содержат пятнадцать уникальных видеофрагментов. Разбив их на кадры и разметив, мы поняли, что классы можно четко разделить на редки и частые. Так, например, класс кухонная мебель (kitchen) встречается 1 раз, когда класс стена без отделки (raw wall) встречается 700+ раз. Это создаёт ощутимые трудности при тренировке. Поэтому, нами принято решение разделить тренировку на тренировку на слабо представленных классах, а затем на всех классах. Для этого нами с помощью сервиса Roboflow подготовленно два разных датасета. Один с подмножеством непопулярных классов, и другой со всеми классами

In [3]:
from roboflow import Roboflow

print("Загрука датасета для первого этапа:\n\n")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
first_stage_dataset = project.version(ROBOFLOW_FIRST_STAGE_DATASET_VERSION).download("yolov8")

# Наш датасет разделен на test/train, поэтому тестовая выборка используется и для валидации
!cp -r {first_stage_dataset.location}/test {first_stage_dataset.location}/valid

Загрука датасета для первого этапа:


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.107, to fix: `pip install ultralytics<=8.0.20`
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format


Extracting Dataset Version Zip to adssadsa-6 in yolov8:: 100%|█████████████████████| 675/675 [00:00<00:00, 15752.97it/s]


In [4]:
from ultralytics import YOLO

print("Отправная версия модели: ", YOLO_INITIAL_MODEL)

model = YOLO(YOLO_INITIAL_MODEL)

Отправная версия модели:  yolov8x.pt


In [5]:
# %env CUDA_VISIBLE_DEVICES=0,1,2

model.train(
    data=f"{first_stage_dataset.location}/data.yaml",
    exist_ok=True,
    batch=64,
    device=0,
    dropout=0.09,
    cls=2,
    label_smoothing=0.17,
    verbose=False,
    pretrained=True,
    imgsz=640,
    name=FIRST_STAGE_EXPERIMENT_NAME,
)

New https://pypi.org/project/ultralytics/8.0.109 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.107 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe, 81100MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/home/alex/v108/adssadsa-6/data.yaml, epochs=100, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=unrepresentative_stage, exist_ok=True, pretrained=True, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.09, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, cl


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      7/100      45.2G      1.408      9.633      1.682         59        640: 100%|██████████| 4/4 [00:02<00:00,  1.37i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140      0.609      0.314      0.265      0.128

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      8/100      45.2G       1.41      8.367       1.66         72        640: 100%|██████████| 4/4 [00:02<00:00,  1.36i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140      0.609       0.22      0.243      0.131

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      9/100      45.2G      1.319      7.786      1.586         61        640: 100%|██████████| 4/4 [


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     27/100      45.1G      1.133      4.192      1.323         73        640: 100%|██████████| 4/4 [00:02<00:00,  1.35i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140      0.433      0.422      0.293      0.157

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     28/100      44.5G      1.172      4.491      1.354         69        640: 100%|██████████| 4/4 [00:02<00:00,  1.33i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140      0.424      0.418      0.279      0.154

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     29/100      45.2G      1.175      4.354      1.324         65        640: 100%|██████████| 4/4 [


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     47/100      45.1G      1.057      3.296      1.285         82        640: 100%|██████████| 4/4 [00:02<00:00,  1.35i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140       0.34      0.242        0.2     0.0707

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     48/100      45.1G      1.083      3.249      1.288         71        640: 100%|██████████| 4/4 [00:02<00:00,  1.35i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140      0.252      0.276      0.228     0.0847

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     49/100      45.1G      1.052      3.303       1.28         70        640: 100%|██████████| 4/4 [


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     67/100      45.1G      0.955      2.663       1.21         77        640: 100%|██████████| 4/4 [00:02<00:00,  1.36i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140      0.523      0.506      0.559      0.365

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     68/100      45.1G          1       2.73      1.224         71        640: 100%|██████████| 4/4 [00:02<00:00,  1.34i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140      0.582      0.505      0.544      0.326

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     69/100      45.1G     0.9357      2.648      1.196         71        640: 100%|██████████| 4/4 [


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     87/100      45.1G     0.8233      2.081      1.135         88        640: 100%|██████████| 4/4 [00:02<00:00,  1.34i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140       0.65      0.825      0.714      0.455

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     88/100      45.1G     0.8492      2.278      1.162         68        640: 100%|██████████| 4/4 [00:02<00:00,  1.35i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00
                   all        110        140      0.619      0.818      0.725      0.465

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     89/100      45.2G     0.7993      1.931      1.127         67        640: 100%|██████████| 4/4 [

In [6]:
model.val(split='val', verbose=True)

Ultralytics YOLOv8.0.107 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe, 81100MiB)
Model summary (fused): 268 layers, 68131272 parameters, 0 gradients
val: Scanning /home/alex/v108/adssadsa-6/valid/labels.cache... 110 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00
                   all        110        140      0.706      0.662      0.747      0.523
               bathtub        110         13          1      0.958      0.995      0.852
                  door        110          3          1      0.988      0.995       0.67
   finished windowsill        110          4          1       0.81      0.995      0.572
               kitchen        110          1          0          0      0.166     0.0995
            raw window        110          1          0          0          0          0
                  sink        110         16      0.973

ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x7f2c7c6de1d0>
fitness: 0.5453167964112731
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.85189,     0.66989,     0.57178,      0.0995,           0,     0.68995,     0.58729,     0.71316])
names: {0: 'bathtub', 1: 'door', 2: 'finished windowsill', 3: 'kitchen', 4: 'raw window', 5: 'sink', 6: 'toilet seat', 7: 'window'}
plot: True
results_dict: {'metrics/precision(B)': 0.7062781442291118, 'metrics/recall(B)': 0.6615361215460414, 'metrics/mAP50(B)': 0.7467772468709613, 'metrics/mAP50-95(B)': 0.5229323019157521, 'fitness': 0.5453167964112731}
save_dir: PosixPath('runs/detect/unrepresentative_stage')
speed: {'preprocess': 1.4978473836725408, 'inference': 6.524064324118875, 'loss': 0.000

# Второй Этап

Второй этап идентичен первому за исключением того, что мы используем все классы из размеченного датасета вместо ограниченного набора.

In [7]:
from roboflow import Roboflow

print("Загрука датасета для второго этапа:\n\n")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
second_stage_dataset = project.version(ROBOFLOW_SECOND_STAGE_DATASET_VERSION).download("yolov8")

# Наш датасет разделен на test/train, поэтому тестовая выборка используется и для валидации
!cp -r {second_stage_dataset.location}/test {second_stage_dataset.location}/valid

Загрука датасета для второго этапа:


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.107, to fix: `pip install ultralytics<=8.0.20`
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:
from ultralytics import YOLO

model = YOLO("runs/detect/unrepresentative_stage/weights/best.pt")

In [12]:
model.train(
    data=f"{second_stage_dataset.location}/data.yaml",
    exist_ok=True,
    batch=80,
    device=0,
    dropout=0.11,
    label_smoothing=0.12,
    verbose=False,
    patience=5,
    pretrained=True,
    imgsz=640,
    name=SECOND_STAGE_EXPERIMENT_NAME,
)

In [5]:
model = YOLO(f"runs/detect/{SECOND_STAGE_EXPERIMENT_NAME}/weights/best.pt")

In [6]:
model.val(split='val', verbose=True)

Ultralytics YOLOv8.0.107 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA A100 80GB PCIe, 81100MiB)
Model summary (fused): 268 layers, 68145717 parameters, 0 gradients
val: Scanning /home/alex/v108/adssadsa-5/valid/labels.cache... 312 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 2, len(boxes) = 1504. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:05<
                   all        312       1504       0.73      0.657      0.716      0.479
               bathtub        312         13      0.923          1      0.995      0.885
                  door        312          3      0.786          1      0.995      0.841
              door-way        312         19      0

ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x7fb055563be0>
fitness: 0.5025899643926537
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.88487,     0.84137,    0.085242,     0.65551,     0.70674,     0.44615,     0.44383,     0.15203,     0.73149,           0,     0.47342,      0.5218,     0.52316,     0.60482,     0.58403,           0,     0.75141,     0.17129,      0.6511,     0.47891,      0.6087,     0.38109,     0.31798])
names: {0: 'bathtub', 1: 'door', 2: 'door-way', 3: 'finished ceiling', 4: 'finished floor', 5: 'finished wall', 6: 'finished windowsill', 7: 'garbage', 8: 'intermediate floor', 9: 'kitchen', 10: 'no-door-doorway', 11: 'radiator', 12: 'raw ceil